# Low-rank approximations for music structure representation and similarity

### Library importing

In [1]:
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp2d
from scipy.sparse.csgraph import laplacian
from scipy.spatial.distance import directed_hausdorff
from scipy.cluster import hierarchy
from scipy.linalg import eigh
from scipy.ndimage import median_filter
import numpy as np
import cv2
from seaborn import clustermap
import sklearn
import librosa
import dill
import sys
import glob
import os
import random
import json
import deepdish as dd
import matplotlib.pyplot as plt
%matplotlib inline

### Dill session

In [ ]:
dill.dump_session('set.db')

In [ ]:
dill.load('shapeset.db')

### Load and compute set of approximation matrices for Da-TACOS

In [12]:
resample_dim = 256 #side size to which to resample Laplacian
kmin = 2 #min components for approximation (>=2)
kmax = 10 #max component for approximation

with open('./da-tacos_metadata/da-tacos_benchmark_subset_metadata.json') as f:
    benchmark_metadata = json.load(f)
count = 0
W_count=0
P_count = 0
all_sets = []
all_WP = []
y = []
for W in benchmark_metadata.keys():
    if len(benchmark_metadata[W].keys()) > 4: #if it contains at least 5 covers
        P_count = 0
        for P in benchmark_metadata[W].keys():
            P_count += 1
            
            #Computations
            try:
                SSM = dd.io.load("./da-tacosSSMs/StructureLaplacian_datacos_crema_" + P + ".h5")['WFused']
            except:
                print("Couldn't load " + P + ".")
                continue

            N = dd.io.load("./da-tacosSSMs/StructureLaplacian_datacos_crema_" + P + ".h5")['N']

            #Construct square matrix from flattened upper triangle
            A = np.zeros((N,N))
            iN = np.triu_indices(N) #return indices for upper-triangle of (N,N) matrix
            for i in range(len(SSM)):
                A[iN[0][i]][iN[1][i]] = SSM[i]
            B = np.transpose(A)
            square_SSM = A+B

            #Resample
            Xindex = np.linspace(0, 1, num=N)
            f = interp2d(Xindex, Xindex, square_SSM.flatten(), kind='linear')
            Xindex_ds = np.linspace(0, 1, num=resample_dim)
            SSM_ds = np.reshape(f(Xindex_ds, Xindex_ds), (resample_dim,resample_dim))

            #Compute the Laplacian
            L = laplacian(SSM_ds, normed=True)

            #Laplacian eigenvalues and eigenvectors
            evals, evecs = eigh(L)

            ##Shape DNA
            #shapeDNA = evals[:30]
            #all_shapeDNA.append(shapeDNA)

            #Hierarchical structure
            evecs = median_filter(evecs, size=(9, 1))
            Cnorm = np.cumsum(evecs**2, axis=1)**0.5
            dist_set = []
            for k in range(kmin, kmax):
                X = evecs[:, :k] / Cnorm[:, k-1:k]
                distance = squareform(pdist(X, metric='euclidean'))
                dist_set.append(distance)

            all_sets.append(dist_set)
            y.append(W)

            #append W and P
            all_WP.append([W, P])

            #plt.matshow()
            #plt.colorbar()
            #plt.show()

            if (P_count >=5):
                break
                
        W_count +=1
        sys.stdout.write("\rLoading %i/100 works." % W_count)
        sys.stdout.flush()
        if (W_count >= 10):
            break
            
all_sets = np.asarray(all_sets)

print("\nLoaded Da-TACOS SMMs.")
print("Data shape:", all_sets.shape)

Loading 10/100 works.
Loaded Da-TACOS SMMs.
Data shape: (50, 8, 256, 256)
